In [1]:
%%time
from functools import partial
from collections import defaultdict
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product
import time

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.postprocessing import *
from video699.screen.semantic_segmentation.evaluation import *

CPU times: user 5.9 s, sys: 1.14 s, total: 7.03 s
Wall time: 19.9 s


In [2]:
detector = FastAIScreenDetector()
method_params = list(detector.methods.keys())
train_params = list(detector.train_params.keys())
all_params = train_params + method_params

base = [True]
base_lower_bounds = [7, 10]
base_upper_bounds = [30, 40, 50, 60, 70]
base_factors = [[0.1, 0.01]]

erode_dilate = [True]
erode_dilate_lower_bounds = [5]
erode_dilate_upper_bounds = [40]
erode_dilate_factors = [[0.1, 0.01]]
erode_dilate_iterations = [40, 100]

ratio_split = [True]
ratio_split_lower_bounds = [0.7, 0.9]
ratio_split_upper_bounds = [1.5]

methods_values = [base] + [erode_dilate] + [ratio_split] + [base_lower_bounds] + [base_upper_bounds] \
        + [base_factors] + [erode_dilate_lower_bounds] + [erode_dilate_upper_bounds] + [erode_dilate_factors] \
        + [erode_dilate_iterations] + [ratio_split_lower_bounds] + [ratio_split_upper_bounds]

batch_size = [8]
resize_factor = [2]
frozen_epochs = [2, 6, 9]
unfrozen_epochs = [3, 7, 10]
frozen_lr = [1e-3]
unfrozen_lr = [slice(1e-4, 2e-4)]

train_params_values = [batch_size] + [resize_factor] + [frozen_epochs] + [unfrozen_epochs] + [frozen_lr] + [unfrozen_lr]

In [3]:
method_settings = list(product(*methods_values))
train_settings = list(product(*train_params_values))
all_lectures = [video.filename for video in ALL_VIDEOS]
all_frames = [frame for video in ALL_VIDEOS for frame in video]
all_frames_grouped_by_videos = {video.filename: [frame for frame in video] for video in ALL_VIDEOS}
test_lectures = ['PB069-D2-20140305.mp4']
test_frames = [frame for lecture in test_lectures for frame in all_frames_grouped_by_videos[lecture]]
actual_detector = AnnotatedSampledVideoScreenDetector()

In [4]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=123)
df_best_models = pd.DataFrame(columns=all_params + ['iou', 'wrong_count'])
for i, split in tqdm(enumerate(kf.split(all_lectures))):
    other_lectures = [all_lectures[index] for index in split[0]]
    test_lectures = [all_lectures[index] for index in split[1]]
    
    # Model selection
    df_all = pd.DataFrame(columns=all_params + ['iou', 'wrong_count', 'kfold_split'])
    
    for train_setting in tqdm(train_settings):
        train_params_dict = dict(zip(train_params, train_setting))
        for j, split in enumerate(kf.split(other_lectures)):
            train_lectures = [other_lectures[index] for index in split[0]]
            valid_lectures = [other_lectures[index] for index in split[1]]
            valid_frames = [frame for lecture in valid_lectures for frame in all_frames_grouped_by_videos[lecture]]

            filtered_by = lambda name: any([lecture in str(name) for lecture in train_lectures + valid_lectures])  \
                            and 'frame' in str(name)
            split_by = lambda name: any([lecture in str(name) for lecture in valid_lectures])
            
            detector = FastAIScreenDetector(train_params=train_params_dict, methods=None, filtered_by=filtered_by,
                                        valid_func=split_by, progressbar=False, device='cuda')
        
            detector.train()
            
            actuals = [actual_detector.detect(frame) for frame in valid_frames]
            sem_preds = detector.semantic_segmentation_batch(valid_frames)
            
            for i, method_setting in enumerate(method_settings):    
                preds = detector.post_processing_batch(sem_preds, valid_frames, dict(zip(method_params, method_setting)))
                wrong_count, ious, _ = evaluate(actuals, preds)
                
                iou_score = np.nanmean(ious)
                wrong_count = len(wrong_count)
                df_all.loc[len(df_all)] = train_setting + method_setting + (iou_score, wrong_count, j)
    
    unhashable_columns = ['frozen_lr', 'unfrozen_lr', 'base_factors', 'erode_dilate_factors']
    df_all[unhashable_columns] = df_all[unhashable_columns].astype(str)
    df_all['wrong_count'] = df_all['wrong_count'].astype(int)
    
    best_params = df_all.groupby(train_params + method_params).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name
    converted_params = []
    for i, par in enumerate(best_params):
        if isinstance(par, np.int64) or isinstance(par, np.float64):
            converted_params.append(par.item())
        else:
            converted_params.append(par)
    best_params = tuple(converted_params)
    
    best_methods = dict(zip(method_params, best_params[-len(method_params):]))
    best_train_params_dict = dict(zip(train_params, best_params[:len(train_params)]))
    best_train_params_dict['frozen_lr'] = eval(best_train_params_dict['frozen_lr'])
    best_train_params_dict['unfrozen_lr'] = eval(best_train_params_dict['unfrozen_lr'])
    best_methods['base_factors'] = eval(best_methods['base_factors'])
    best_methods['erode_dilate_factors'] = eval(best_methods['erode_dilate_factors'])
    
    
    filtered_by = lambda name: 'frame' in str(name)
    split_by = lambda name: any([lecture in str(name) for lecture in test_lectures])
    
    best_detector = FastAIScreenDetector(train_params=best_train_params_dict, methods=best_methods, 
                                         filtered_by=filtered_by, valid_func=split_by, progressbar=True, device='cuda')
    best_detector.train()

    actuals = [actual_detector.detect(frame) for frame in valid_frames]
    preds = [best_detector.detect(frame) for frame in valid_frames]
    wrong_count, iou, _ = evaluate(actuals, preds)
    iou_score = np.nanmean(ious)
    wrong_count = len(wrong_count)
    df_best_models.loc[len(df_best_models)] = train_setting + method_setting + (iou_score, wrong_count)
    df_best_models.to_csv('cross_validation_results.csv', index=False)
df_best_models.to_csv('cross_validation_results.csv')

0it [00:00, ?it/s]
  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [07:40<1:01:24, 460.55s/it]


 22%|██▏       | 2/9 [19:33<1:02:34, 536.30s/it]


 33%|███▎      | 3/9 [34:46<1:04:55, 649.25s/it]


 44%|████▍     | 4/9 [46:45<55:51, 670.21s/it]  

...


100%|██████████| 9/9 [2:27:14<00:00, 981.64s/it] 


epoch,train_loss,valid_loss,acc,dice,iou,time
0,0.337639,0.166160,0.904206,0.834766,0.759246,00:17
1,0.191064,0.079999,0.979892,0.951992,0.919810,00:17


epoch,train_loss,valid_loss,acc,dice,iou,time
0,0.084899,0.053519,0.983677,0.958577,0.932177,00:17
1,0.060548,0.032533,0.989308,0.966680,0.947760,00:17
2,0.047564,0.064651,0.980075,0.955695,0.926819,00:17
3,0.042504,0.022390,0.990919,0.967391,0.949460,00:17
4,0.034587,0.086912,0.977897,0.950310,0.917816,00:17
5,0.025196,0.023109,0.992688,0.972955,0.959688,00:17
6,0.021220,0.017067,0.994430,0.977163,0.967890,00:17
7,0.016796,0.011445,0.995355,0.978281,0.970058,00:17
8,0.014509,0.010906,0.995517,0.978336,0.970179,00:17
9,0.013038,0.010151,0.995869,0.979120,0.971711,00:17


5it [12:20:44, 8888.85s/it]

CPU times: user 8h 44min 5s, sys: 2h 14min 28s, total: 10h 58min 33s
Wall time: 12h 20min 44s
